In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import tqdm

import torchvision
from torchvision import datasets, transforms
import videotransforms


import numpy as np

from pytorch_i3d import InceptionI3d

from charades_dataset_full import Charades as Dataset

In [ ]:
test_transforms = transforms.Compose([videotransforms.CenterCrop(224)])

In [ ]:

import os

split = '/opt/ml/input/data/test.json'
root = '/opt/ml/input/data'
mode = 'rgb'
batch_size = 1
save_dir = '/opt/ml/input/data'
load_model = '/opt/ml/input/code/pytorch-i3d/models/rgb_charades.pt'

dataset = Dataset(split, 'training', root, mode, test_transforms, num=-1, save_dir=save_dir)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

val_dataset = Dataset(split, 'testing', root, mode, test_transforms, num=-1, save_dir=save_dir)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=False)    

dataloaders = {'train': dataloader, 'val': val_dataloader}
datasets = {'train': dataset, 'val': val_dataset}

# setup the model
if mode == 'flow':
    i3d = InceptionI3d(400, in_channels=2)
else:
    i3d = InceptionI3d(400, in_channels=3)
i3d.replace_logits(157)
i3d.load_state_dict(torch.load(load_model))
i3d.cuda()

# for phase in ['train', 'val']:
# try:

for phase in ['val']:
    i3d.train(False)  # Set model to evaluate mode
            
    tot_loss = 0.0
    tot_loc_loss = 0.0
    tot_cls_loss = 0.0
                
    # Iterate over data.
    for data in dataloaders[phase]:
        # get the inputs
        inputs, labels, name = data
    
        if os.path.exists(os.path.join(save_dir, name[0]+'.npy')):
            continue

        b,c,t,h,w = inputs.shape
        if t > 1600:
            features = []
            for start in range(1, t-56, 1600):
                end = min(t-1, start+1600+56)
                start = max(1, start-48)
                # ip = Variable(torch.from_numpy(inputs.numpy()[:,:,start:end]).cuda(), volatile=True)
                ip = Variable(torch.from_numpy(inputs.numpy()[:,:,start:end]).cuda())
                features.append(i3d.extract_features(ip).squeeze(0).permute(1,2,3,0).data.cuda().numpy())
            np.save(os.path.join(save_dir, name[0]), np.concatenate(features, axis=0))
        else:
            # wrap them in Variable
            inputs = Variable(inputs.cuda())
            features = i3d.extract_features(inputs)
            np.save(os.path.join(save_dir, name[0]), features.squeeze(0).permute(1,2,3,0).data.cpu().numpy())

동영상을 사진으로 변환

In [ ]:
import cv2

cap = cv2.VideoCapture('/opt/ml/input/data/test_video.mp4')
FPS = cap.get(5)
frame = cap.get(7)
count = 1

for i in range(1, int(frame)):
    success, img = cap.read()
    
    if not success:
        break

    if i % int(FPS / 1.5) == 0:
        cv2.imwrite(f"/opt/ml/input/data/test_video/test_video"+'-'+str(count).zfill(6)+'.jpg', img)
        count+=1


In [ ]:
import numpy as np
result =np.load('/opt/ml/input/data/test_video.npy')
result = result.reshape(-1, 1024)

result.shape


In [ ]:
np.save('/opt/ml/input/data/new_test.npy', result)

In [ ]:
test =np.load('/opt/ml/input/data/new_test.npy')
test.shape